In [ ]:
import warnings
warnings.filterwarnings(action='ignore')  # 실행 중 표시되는 경고를 무시합니다 (optional)

# 필요한 라이브러리들을 불러와봅시다
import qiskit
import qiskit_ibm_runtime
import qiskit_machine_learning
import qiskit_optimization

import qiskit.tools.jupyter
%qiskit_version_table  # import 된 qiskit 라이브러리들의 버전을 표시합니다

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_multivector

In [ ]:
# 덧셈을 수행하는 회로입니다
circuit = QuantumCircuit(4)
circuit.cx(0, 2)
circuit.cx(1, 2)
circuit.ccx(0, 1, 3)
circuit.draw('mpl')

In [ ]:
# 양자 텔레포트(Quantum Teleportation)에 대해 알아봅시다
# 양자 텔레포트 프로토콜을 통해 전송할 임의의 큐비트를 설정했습니다
state = Statevector(np.array([1.+0.j, 1.-1.j])/np.sqrt(3))
plot_bloch_multivector(state)

In [ ]:
# 양자 텔레포트에서 관측과 조건부 게이트 대신 CX와 CZ 게이트를 이용해 회로를 만들었습니다
circuit = QuantumCircuit(3)
circuit.initialize(state, qubits=[0])
circuit.h(1)
circuit.cx(1, 2)
circuit.barrier()
circuit.cx(0, 1)
circuit.h(0)
circuit.barrier()
circuit.cz(0, 2)
circuit.cx(1, 2)
circuit.draw('mpl')

In [ ]:
# 회로의 결과로 얻어지는 큐비트의 상태를 Bloch sphere로 나타냅니다
# 처음 qubit 0 의 상태가 qubit 2 로 옮겨진 것을 확인할 수 있습니다
state = Statevector.from_instruction(circuit)
plot_bloch_multivector(state)

In [ ]:
# 양자 텔레포트 프로토콜 중 qubit 0과 1을 관측했을 때의 확률 분포는 어떤지 알아봅시다
# 이번에는 qubit 0과 1을 관측하도록 회로를 작성했습니다
circuit = QuantumCircuit(3, 2)
circuit.initialize(state, qubits=[0])
circuit.h(1)
circuit.cx(1, 2)
circuit.barrier()
circuit.cx(0, 1)
circuit.h(0)
circuit.barrier()
circuit.measure([0, 1], [0, 1])
circuit.draw('mpl')

In [ ]:
from qiskit.primitives import Sampler

# qiskit.primitives에서 Sampler를 불러와 시뮬레이션을 수행합니다
sampler = Sampler()  # Sampler를 초기화합니다
job = sampler.run(circuit)  # Sampler에 회로를 넣어 실행합니다
result = job.result()  # 실행된 job의 결과를 불러옵니다
quasi_dists = result.quasi_dists[0]  # 결과에서 준확률분포(quasi_dists)를 불러옵니다
print(quasi_dists)  # 준확률분포를 출력합니다

In [ ]:
from qiskit.visualization import plot_histogram

# 히스토그램 형태로 준확률분포를 그립니다
# qubit 0과 1은 서로 아무런 관계가 없으므로 서로의 XX방향 ZZ방향 관계를 측정하는 Bell measurement 결과는 완전히 무작위로 나타납니다
plot_histogram(quasi_dists)

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Options, Session

# Qiskit Runtime에 대해 알아봅시다
# Qiskit Runtime은 IBM Quantum 계정을 불러와 클라우드 상의 시뮬레이터와 양자컴퓨터에 연결할 수 있게 해줍니다
# Qiskit Runtime Service를 불러오는 방법에는 두 가지가 있습니다

# 첫 번째 방법: 서비스에 채널, 토큰, 인스턴스 정보를 입력해 불러오기
service = QiskitRuntimeService(channel='ibm_quantum', token='Your_token', instance='Your_instance')

# # 두 번째 방법: 채널, 토큰, 인스턴스 정보를 저장해두고 불러오기
# # 한 번 save_account 함수로 정보를 저장해두면 그 뒤로는 QiskitRuntimeService() 만으로 서비스를 초기화할 수 있습니다
# QiskitRuntimeService.save_account(channel='ibm_quantum', token='Your_token', instance='Your_instance')
# service = QiskitRuntimeService()

In [ ]:
# 접속한 서비스에서 사용 가능한 양자컴퓨터와 시뮬레이터 목록을 불러옵니다
service.backends()

In [ ]:
# 원하는 백엔드를 선택합니다
backend = service.backend("ibm_kyoto")
backend.status()  # 선택한 백엔드의 상태를 출력합니다

In [ ]:
# 불러온 서비스와 백엔드에 세션을 잡아둡니다
session = Session(service, backend)

In [ ]:
# Sampler primitive에 session을 넣어 초기화해주면 주어진 세션 ID를 사용하여 계산을 수행합니다
sampler = Sampler(session=session)

In [ ]:
# 실행하고 싶은 회로를 임의로 만들었습니다
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])
qc.draw('mpl')

In [ ]:
# sampler를 이용해 회로 계산을 수행합니다
# job이 생성되어 서버로 전송됩니다
job = sampler.run(qc)

In [ ]:
# job의 실행 상태를 확인합니다
# job은 대기 중, 계산 중, 성공, 실패 등의 상태에 있을 수 있습니다
job.status()

In [ ]:
# job이 완료되면 이전과 같이 results() 함수로 결과를 받아오고 quasi_dists 데이터를 확인할 수 있습니다
results = sampler.results()
quasi_dists = results.quasi_dists[0]
plot_histogram(quasi_dists)

In [ ]:
# Qiskit Runtime Option을 활용하여 error mitigation을 적용해봅시다
# Option은 Options 클래스를 불러와 설정할 수 있습니다
# 설정할 수 있는 다양한 옵션 중 error mitigation을 설정하는 argument는 resilience_level 입니다
# 각 resilience level에서 활성화되는 error mitigation 종류는 다음 document에서 확인할 수 있습니다
# https://docs.quantum.ibm.com/run/configure-error-mitigation
options = Options(resilience_level=1)
sampler = Sampler(session=session, options=options)

In [ ]:
# 세션 사용시에는 항상 마지막에 세션을 닫아주셔야 합니다
session.close()

In [ ]:
# 한 번에 여러 작업을 수행하고 세션을 바로 닫을 때에는 with 메소드를 이용해 세션이 자동으로 닫히도록 설정할 수도 있습니다
with Session(service=service, backend=backend) as session:
    sampler = Sampler(session=session, options=options)
    quasi_dists = sampler.run(qc).results().quasi_dists[0]  # 한 번에 준확률분포까지 받아올 수도 있습니다
    # 이 경우에는 with문이 끝나면 자동으로 세션이 닫히게 됩니다